# cheking what segments are good to go

In [91]:
PATHFunciones = "/home/isabel/Dropbox/Proyectos/Papers/Romero-2022";
HD = "/run/media/isabel/7bb4008d-08df-440b-9635-60c2187d8dfb/";
PATHMain = string( HD, "Data/Hipocampo/Espontanea/01-06-2022/Phase_10");

push!( LOAD_PATH, PATHFunciones );
using JuliaTools
using Revisadas
using JLD

PATHInfo = joinpath( PATHMain , "Info" ); 
FILEVars = joinpath( PATHInfo, "variablesBRW.jld" );
PATHNoSat = joinpath( PATHMain, "NoSats" ); 

Variables = load( FILEVars )[ "Variables" ];
nChs = length( Variables[ "Layout" ] ); 
FILESNoSat = searchdir( PATHNoSat, "jld" ); 
FILEFoto = "/home/isabel/Dropbox/01-06-2022/Fotos/Rebanada.png";

Aqui se sacan parametros de cada segmento

In [3]:
nChs = length( Variables[ "Layout" ] ); # total number of channels (...4096)
p = 0.90;

sparsity_matrix = zeros( length( FILESNoSat ), 3 ); # Array n segments rows x 3 columns
variabilidades = zeros( nChs, length( FILESNoSat ) ); # Array 4096 rows x n segments columns

@time for i = 1:length( FILESNoSat );

    FILENoSat = FILESNoSat[ i ];
    
    data = Float64.( load( FILENoSat )[ "data" ] ); 

    V = zeros( Int, nChs ); # preallocation

    for j = 1:nChs; V[ j ] = length( unique( data[ j, : ] ) ); end

    xmean, Δx, C1, C2 = MeanΔxCI( V, p );
    
    AboveCI = findall( V .>= C1 ); 
    
    grupos, loose = Get_Groups( AboveCI );
    
    # Density, N channel above CI, Sparsity
    sparsity_matrix[ i, : ] = [ Density( grupos ), length( AboveCI ), Sparsity( length( grupos ), nChs ) ]
    
    # V vector of each segment into an array
    variabilidades[ :, i ] = V;
    
end

save( joinpath( PATHInfo, "sparsity_matrix.jld" ), "data", sparsity_matrix );

save( joinpath( PATHInfo, "variabilidades.jld" ), "data", variabilidades );

2082.787390 seconds (26.39 G allocations: 709.246 GiB, 2.48% gc time, 0.07% compilation time)


De aqui se selecciona el cacho que se va a analizar

In [16]:
using Plots
using Statistics

X = findall( maximum( mean( variabilidades, dims = 1 ) ) .== mean( variabilidades, dims = 1 ) )[ 1 ][ 2 ];

BIN = load( FILESNoSat[ X ] );

V = variabilidades[ :, X ];

xmean, Δx, C1, C2 = MeanΔxCI( V, p );

Above = setdiff( 1:nChs, findall( C1 .>= V ) ); 

F1 = Zplot( Z0( Above ), :greys );
F2 = Zplot( Z0( BIN[ "Seleccion" ] ), :greys );
W = variabilidades[ :, 36 ];
Z = reverse( reshape( W, Variables[ "NRows" ], Variables[ "NCols" ] )', dims = 1 ); 

FigureTest = Zplot( Z, :bwr );

plot( F1, F2, FigureTest, layout = ( 1, 3 ), wsize = ( 900, 400 ) );

savefig( "nVoltajes.svg" );

Identificado el cacho, se hace todo el procedimiento solo para ese segmento

In [86]:
limite = 550; # μm 

channels = 1:nChs;

Canales = BIN[ "Seleccion" ];

Canales = sort( Int.( Canales ) );

Z = Z0( Canales ); F1 = Zplot( Z, :greys );

grupos, canales_solos = Get_Groups( Canales );


# ----------------------------------

# Si hay mancha hay que quitarla o identificarla...

mancha = sort( length.( grupos ) )[ end - 1 ];

Todos = vcat( ( grupos[ length.( grupos ) .!= mancha ] )...);

Z = Z0( Todos ); F2 = Zplot( Z, :greys );

nChsCA = maximum( length.( grupos ) );

CA = grupos[ length.( grupos ) .== nChsCA ][ 1 ];

Z = Z0( CA ); F3 = Zplot( Z, :greys );

# buscar el miembro mas cercano del grupo CA

sueltos = setdiff( Todos, CA );

CACoo = Index2Coordinates( CA ); sueltosCoo = Index2Coordinates( sueltos );

D_Sueltos = Array{ Float64 }( undef, length( CA )*length( sueltos ), 2 ); fill!( D_Sueltos, 0 );

for i = 1:length( sueltos )
    
    for j = 1:length( CA )

        dist =
            sqrt( ( sueltosCoo[ i, 2 ] - CACoo[ j, 2 ] )^2
                + ( sueltosCoo[ i, 3 ] - CACoo[ j, 3 ] )^2 );
        
        D_Sueltos[ ( ( i - 1 )*length( CA ) + j ), : ] = [ sueltosCoo[ i, 1 ], ( dist * 42 ) ];
        
    end
end

SueltosSeleccionados = D_Sueltos[ :, 1 ][ findall( D_Sueltos[ :, 2 ] .<= limite ) ];

SueltosSeleccionados = Int.( unique( SueltosSeleccionados ) );
 
Z = Z0( union( CA, SueltosSeleccionados ) ); F4 = Zplot( Z, :greys );

In [87]:
# Rellenar CA

oldCA = copy( CA ); huecos = [ ]; Nhuecos = 6;

while !isempty( huecos ) || length( CA ) == length( oldCA )
    
    noCA = setdiff( collect( channels ), CA );

    noCAnoSueltos = setdiff( noCA, SueltosSeleccionados );

    nVecinos = zeros( Int, length( noCAnoSueltos ), 2 );

    for i = 1:length( noCAnoSueltos )
        
        _, v = JuliaTools.neighborgs( noCAnoSueltos[ i ], 1 );

        nVecinos[ i, : ] = [ noCAnoSueltos[ i ], length( intersect( CA, v ) ) ];
    end

    huecos = nVecinos[ :, 1 ][ findall( nVecinos[ :, 2 ] .>= Nhuecos ) ];

    CA = vcat( CA, huecos );
    
end

Z = Z0( CA ); F5 = Zplot( Z, :greys );

In [88]:
# Rellenar Sueltos

oldSueltos = copy( SueltosSeleccionados ); #SueltosSeleccionados = oldSueltos;

huecos = [ ]; Nhuecos = 5;

while !isempty( huecos ) || length( SueltosSeleccionados ) == length( oldSueltos )
    
    noSueltos = setdiff( collect( channels ), SueltosSeleccionados );

    noSueltosnoCA = setdiff( noSueltos, CA );

    nVecinos = zeros( Int, length( noSueltosnoCA ), 2 );

    for i = 1:length( noSueltosnoCA )

        _, v = JuliaTools.neighborgs( noSueltosnoCA[ i ], 1 );

        nVecinos[ i, : ] = [ noSueltosnoCA[ i ], length( intersect( SueltosSeleccionados, v ) ) ];
    end

    huecos = nVecinos[ :, 1 ][ findall( nVecinos[ :, 2 ] .>= Nhuecos ) ];

    SueltosSeleccionados = vcat( SueltosSeleccionados, huecos );
    
end

In [94]:
Z = Z0( SueltosSeleccionados ); F6 = Zplot( Z, :greys );

Z = Z0( union( CA, SueltosSeleccionados ) ); F7 = Zplot( Z, :greys );

F8 = plot( load( FILEFoto ),
        aspect_ratio = 1,
        axis = ( [ ], false ),
        wsize = ( 400, 400 ),
        titlefont = ( 10, "arial" ) );

CASueltos = union( CA, SueltosSeleccionados );

grupos, canales_solos = Get_Groups( CASueltos );

Z = zeros( Int, nChs );

for i = 1:length( grupos )
    Z[ grupos[ i ] ] .= i + 1;
end

Z[ canales_solos ] .= 1;

Z = reverse( reshape( Z, Variables[ "NCols" ], Variables[ "NRows" ] )', dims = 1 );

F11 = Zplot( Z, :twilight );

In [96]:
# seleccionar los 2 grupos mas grandes sin CA y eliminar las islas que estan lejos

GDparcial = sort( length.( grupos ) )[ end-2:end-1 ]; # 2 grupos grandes que no son CA

GDparcial = vcat( union( grupos[ length.( grupos ) .== GDparcial[ 1 ] ], 
        grupos[ length.( grupos ) .== GDparcial[ 2 ] ] )... );

aux = grupos[length.(grupos) .!= maximum( length.(grupos) )];

aux = setdiff( vcat(aux...), GDparcial );

GDpotenciales = union( aux, canales_solos );

D_GDpotenciales = Array{ Float64 }( undef, length( GDparcial )*length( GDpotenciales ), 2 ); 

fill!( D_GDpotenciales, 0 );

GDparcialCoo = Index2Coordinates( Int.( GDparcial ) );

GDpotencialesCoo = Index2Coordinates( Int.( GDpotenciales ) );

for i = 1:length( GDpotenciales )
    
    for j = 1:length( GDparcial )

        dist =
            sqrt( ( GDpotencialesCoo[ i, 2 ] - GDparcialCoo[ j, 2 ] )^2
                + ( GDpotencialesCoo[ i, 3 ] - GDparcialCoo[ j, 3 ] )^2 );
        
        D_GDpotenciales[ ( ( i - 1 )*length( GDparcial ) + j ), : ] = [ 
            GDpotencialesCoo[ i, 1 ], ( dist * 42 ) ];
        
    end
end

limite = 900; #μm

GDpotencialesSeleccionados = D_GDpotenciales[ :, 1 ][ findall( D_GDpotenciales[ :, 2 ] .<= limite ) ];

GDpotencialesSeleccionados = Int.( unique( GDpotencialesSeleccionados ) );

GD = unique( union( GDpotencialesSeleccionados, GDparcial ) );

Z = zeros( Int, nChs ); Z[ GD ] = Z[ GD ] .+ 2; Z[ CA ] = Z[ CA ] .+ 1;

Z = reverse( reshape( Z, Variables[ "NCols" ], Variables[ "NRows" ] )', dims = 1 );

F10 = Zplot( Z, :ice );

In [99]:
Variables[ "CA" ] = CA; Variables[ "GD" ] = GD; #save( FILEVars, "Variables", Variables );

In [107]:
# -------------------------------------------------------------------------------------------------------- #

PATHSpikesCA = joinpath( PATHMain, "CAEspigas" ); mkpath( PATHSpikesCA );
PATHSpikesGD = joinpath( PATHMain, "GDEspigas" ); mkpath( PATHSpikesGD );

SamplingRate = Variables[ "SamplingRate" ][ ];


# Para hacer los filtrados y deteccion de espigas
window = 5; # ms tamaño de la ventana
displace = 1; # ms desplazamiento de la ventana
stokes = 0.5; # ms distancia mínima entre evento y evento detectado
stdmin = 4; # umbral de deteccion
w_pre = 1; # ms
w_post = 2; # ms

# Centrando las espigas
Icut = 0.5; # ms cuantos puntos
Lcut = 1; # ms

# preparando para filtrar
lF = 300; # Hz Frequencia limite izquierdo para el pasa bandas
HF = 3000; # Hz Frecuencia limite derecho para el pasa bandas
fac = 10; # Hz para la atenuacion entre las bandas
NYQ = floor( Int, SamplingRate/2 );
order = Int( floor( ( SamplingRate / lF ) / 5 ) );

# limite superior de deteccion
Vmax = 300; # μV

parametros = Dict(
        "window" => JuliaTools.ms2frames( window, SamplingRate ),
        "displace" => JuliaTools.ms2frames( displace, SamplingRate ),
        "stokes" => JuliaTools.ms2frames( stokes, SamplingRate ),
        "w_pre" => JuliaTools.ms2frames( w_pre, SamplingRate ),
        "w_post" => JuliaTools.ms2frames( w_post, SamplingRate ),
        "Icut" => JuliaTools.ms2frames( Icut, SamplingRate ),
        "Lcut" => JuliaTools.ms2frames( Lcut, SamplingRate ),
        "stdmin" => stdmin,
        "lF" => lF,
        "HF" => HF,
        "fac" => fac,
        "Vmax" => Vmax,
        "NYQ" => NYQ,
        "order" => order
    );

In [104]:
MatrixCA = BIN[ "data" ][ CA, : ]; mapa_indexCA = hcat( 1:length( CA ), CA )

MatrixGD = BIN[ "data" ][ GD, : ]; mapa_indexGD = hcat( 1:length( GD ), GD )

In [110]:
# For CA

for A = 1:length( mapa_indexCA[ :, 1 ] );
    
    Ncanal = Int.( mapa_indexCA[ A, 2 ] );
    
    canal = MatrixCA[ A, : ];
    
    SpikesMat( Ncanal, canal, parametros, Variables, PATHSpikesCA )
        
end

# For GD

for A = 1:length( mapa_indexGD[ :, 1 ] );
    
    Ncanal = Int.( mapa_indexGD[ A, 2 ] );
    
    canal = MatrixGD[ A, : ];
    
    SpikesMat( Ncanal, canal, parametros, Variables, PATHSpikesGD )
        
end

# -------------------------------------------------------------------------------------------------------- #

espigas del canal -> 1209 listas
espigas del canal -> 1210 listas
espigas del canal -> 1272 listas
espigas del canal -> 1337 listas
espigas del canal -> 1338 listas
espigas del canal -> 1339 listas
espigas del canal -> 1340 listas
espigas del canal -> 1401 listas
espigas del canal -> 1402 listas
espigas del canal -> 1403 listas
espigas del canal -> 1466 listas
espigas del canal -> 1467 listas
espigas del canal -> 1527 listas
espigas del canal -> 1528 listas
espigas del canal -> 1529 listas
espigas del canal -> 1530 listas
espigas del canal -> 1531 listas
espigas del canal -> 1532 listas
espigas del canal -> 1589 listas
espigas del canal -> 1590 listas
espigas del canal -> 1592 listas
espigas del canal -> 1593 listas
espigas del canal -> 1594 listas
espigas del canal -> 1595 listas
espigas del canal -> 1650 listas
espigas del canal -> 1651 listas
espigas del canal -> 1653 listas
espigas del canal -> 1654 listas
espigas del canal -> 1655 listas
espigas del canal -> 1656 listas
espigas de

espigas del canal -> 2788 listas
espigas del canal -> 2789 listas
espigas del canal -> 2790 listas
espigas del canal -> 2791 listas
espigas del canal -> 2792 listas
espigas del canal -> 2793 listas
espigas del canal -> 2794 listas
espigas del canal -> 2795 listas
espigas del canal -> 2796 listas
espigas del canal -> 2797 listas
espigas del canal -> 2798 listas
espigas del canal -> 2799 listas
espigas del canal -> 2800 listas
espigas del canal -> 2801 listas
espigas del canal -> 2802 listas
espigas del canal -> 2803 listas
espigas del canal -> 2804 listas
espigas del canal -> 2805 listas
espigas del canal -> 2806 listas
espigas del canal -> 2807 listas
espigas del canal -> 2848 listas
espigas del canal -> 2849 listas
espigas del canal -> 2850 listas
espigas del canal -> 2851 listas
espigas del canal -> 2852 listas
espigas del canal -> 2853 listas
espigas del canal -> 2854 listas
espigas del canal -> 2855 listas
espigas del canal -> 2856 listas
espigas del canal -> 2857 listas
espigas de